In [1]:
import random

num_students = 450
num_profiles = 8
min_num_groups = 1
min_group_size = 20
max_group_size = 30
max_num_groups = 5    

students = list(range(num_students))
profiles = list(range(num_profiles))

# Формируем квоты для профилей
quotas = [max_group_size * max_num_groups for _ in range(num_profiles)]

# Генерация меток
student_labels = {
    'is_target': [0] * num_students,
    'is_contract': [0] * num_students,
    'debts': [],
    'rating': []
}

# Генерация меток целевых студентов
num_target_students = int(0.05 * num_students)
target_students = random.sample(students, num_target_students)

for target in target_students:
    student_labels['is_target'][target] = 1

# Генерация меток контрактников
num_contract_students = int(0.3 * num_students)
contract_students = random.sample(students, num_contract_students)

for contract in contract_students:
    student_labels['is_contract'][contract] = 1

# Генерация академических задолженностей
def generate_debts():
    return random.choices(range(8), weights=[10, 5, 3, 3, 2, 2, 1, 1], k=1)[0]

student_labels['debts'] = [generate_debts() for _ in students]

# Генерация баллов рейтинга
student_labels['rating'] = [random.randint(0, 500) for _ in students]

for i in range(num_students):
    print(f"Студент {i}: Целевой - {student_labels['is_target'][i]}, "
          f"Контрактник - {student_labels['is_contract'][i]}, "
          f"Долги - {student_labels['debts'][i]}, "
          f"БРС - {student_labels['rating'][i]}")

Студент 0: Целевой - 0, Контрактник - 0, Долги - 4, БРС - 469
Студент 1: Целевой - 0, Контрактник - 0, Долги - 6, БРС - 150
Студент 2: Целевой - 0, Контрактник - 0, Долги - 0, БРС - 98
Студент 3: Целевой - 0, Контрактник - 1, Долги - 3, БРС - 171
Студент 4: Целевой - 0, Контрактник - 0, Долги - 1, БРС - 159
Студент 5: Целевой - 0, Контрактник - 0, Долги - 3, БРС - 436
Студент 6: Целевой - 0, Контрактник - 0, Долги - 0, БРС - 264
Студент 7: Целевой - 1, Контрактник - 0, Долги - 5, БРС - 389
Студент 8: Целевой - 0, Контрактник - 0, Долги - 0, БРС - 270
Студент 9: Целевой - 0, Контрактник - 1, Долги - 0, БРС - 163
Студент 10: Целевой - 0, Контрактник - 0, Долги - 2, БРС - 389
Студент 11: Целевой - 0, Контрактник - 0, Долги - 0, БРС - 330
Студент 12: Целевой - 0, Контрактник - 0, Долги - 0, БРС - 230
Студент 13: Целевой - 0, Контрактник - 1, Долги - 2, БРС - 192
Студент 14: Целевой - 0, Контрактник - 1, Долги - 3, БРС - 255
Студент 15: Целевой - 0, Контрактник - 0, Долги - 4, БРС - 45
Студ

In [2]:
# Генерация предпочтений студентов
preferences_s = {}
for student in students:
    if student_labels['debts'][student] > 5:
        preferences_s[student] = []  # Равнодушие к профилям, пустой список
    else:
        preferences_s[student] = random.sample(profiles, k=num_profiles)  # Выбор профилей

# Генерация предпочтений профилей
preferences_p = {}
for prof in profiles:
    target_students_sorted = sorted(
        [s for s in students if student_labels['is_target'][s] == 1],
        key=lambda s: -student_labels['rating'][s]
    )

    other_students = sorted(
        [s for s in students if student_labels['is_target'][s] == 0 and student_labels['debts'][s] <= 5],
        key=lambda s: (-student_labels['rating'][s], -student_labels['is_contract'][s])
    )

    debt_students = sorted(
        [s for s in students if student_labels['debts'][s] > 5],
        key=lambda s: -student_labels['rating'][s]
    )

    # Формируем полные предпочтения для профиля
    preferences_p[prof] = target_students_sorted + other_students + debt_students

# Подсчет студентов, которые поставили профиль первым приоритетом
priority_count = {prof: 0 for prof in profiles}
target_priority_count = {prof: 0 for prof in profiles}

for student in students:
    if preferences_s[student]:  # Проверяем, есть ли предпочтения
        first_preference = preferences_s[student][0]
        priority_count[first_preference] += 1  # Увеличиваем счетчик для данного профиля

        if student_labels['is_target'][student] == 1 and first_preference == preferences_s[student][0]:
            target_priority_count[first_preference] += 1  # Увеличиваем счетчик целевых студентов

# Определение обязательных профилей
mandatory_profiles = [
    prof for prof in profiles 
    if target_priority_count[prof] > 4 or priority_count[prof] > 60
]

# Количество обязательных профилей
num_mandatory_profiles = len(mandatory_profiles)

# Объем резерва
reserve_size = num_mandatory_profiles * min_num_groups * min_group_size
reserve_students = []

# Список студентов без предпочтений
students_without_preferences = []
for student in students:
    if not preferences_s[student]:
        students_without_preferences.append(student)

# Список оставшихся нецелевых студентов
remaining_students = []
for student in students:
    if student_labels['is_target'][student] == 0:
        remaining_students.append(student)

# Заполнение резерва студентами без предпочтений
students_without_preferences_sorted = sorted(
    students_without_preferences, 
    key=lambda s: student_labels['rating'][s]
)

for student in reversed(students_without_preferences_sorted):
    if len(reserve_students) < reserve_size:
        reserve_students.append(student)
    else:
        break

# Заполнение резерва оставшимися нецелевыми студентами
remaining_students_sorted = sorted(
    remaining_students,
    key=lambda s: student_labels['rating'][s]
)

for student in reversed(remaining_students_sorted):
    if len(reserve_students) < reserve_size:
        reserve_students.append(student)
    else:
        break

# Вывод результатов
for prof in profiles:
    print(f"Профиль {prof}: Студенты, поставившие первым приоритетом: {priority_count[prof]}, "
          f"целевиков: {target_priority_count[prof]}")

print(f"Размер резерва студентов: {len(reserve_students)}")
print(f"Резерв студентов: {reserve_students}")

Профиль 0: Студенты, поставившие первым приоритетом: 49, целевиков: 2
Профиль 1: Студенты, поставившие первым приоритетом: 52, целевиков: 2
Профиль 2: Студенты, поставившие первым приоритетом: 53, целевиков: 4
Профиль 3: Студенты, поставившие первым приоритетом: 52, целевиков: 3
Профиль 4: Студенты, поставившие первым приоритетом: 56, целевиков: 8
Профиль 5: Студенты, поставившие первым приоритетом: 57, целевиков: 1
Профиль 6: Студенты, поставившие первым приоритетом: 48, целевиков: 1
Профиль 7: Студенты, поставившие первым приоритетом: 46, целевиков: 0
Размер резерва студентов: 20
Резерв студентов: [173, 167, 114, 170, 328, 274, 96, 379, 28, 185, 323, 300, 448, 159, 145, 307, 232, 236, 121, 141]


In [3]:
def gale_shapley(num_students, num_profiles, quotas, preferences_s, preferences_p, mandatory_profiles, reserve_students):
    # Инициализация
    profiles_matches = {prof: [] for prof in range(num_profiles)}  # Сопоставления профилей
    student_matches = {student: None for student in range(num_students)}  # Сопоставления студентов
    reserve_set = set(reserve_students)  # Студенты в резерве
    # Убираем студентов из резерва из remaining_students_after_distribution
    free_students = [s for s in students if s not in reserve_set]  
    proposed = {student: 0 for student in range(num_students)}  # Счетчик предложений

    while free_students:
        # Вытаскиваем свободного студента
        student = free_students[0]
        student_pref = preferences_s[student]

        # Находим наиболее предпочтительный профиль, которому студент еще не делал предложение
        prof_index = proposed[student]

        if prof_index < len(student_pref):  # Здесь используем длину предпочтений, чтобы не сломаться на студентах с пустыми списками
            prof = student_pref[prof_index]
            proposed[student] += 1

            # Если профиль свободен, совмещаем
            if len(profiles_matches[prof]) < quotas[prof]:
                profiles_matches[prof].append(student)
                profiles_matches[prof].sort(key=preferences_p[prof].index)
                student_matches[student] = prof
                free_students.pop(0)

                # Проверяем с обязательным профилем и обрабатываем резерв
                if prof in mandatory_profiles and reserve_students:
                    student_from_reserve = reserve_students.pop(0)  # Берем первого студента из резерва
                    free_students.append(student_from_reserve)  # Добавляем студента в список свободных
            else:
                # Если профиль занят, проверяем, хочет ли он заменить текущего худшего студента
                current_student = profiles_matches[prof][-1]
                prof_pref = preferences_p[prof]

                if prof_pref.index(student) < prof_pref.index(current_student):
                    profiles_matches[prof].pop()
                    profiles_matches[prof].append(student)
                    profiles_matches[prof].sort(key=prof_pref.index)
                    student_matches[student] = prof
                    student_matches[current_student] = None
                    free_students.pop(0)
                    free_students.append(current_student)

                    # Проверяем с обязательным профилем и обрабатываем резерв
                    if prof in mandatory_profiles and reserve_students:
                        student_from_reserve = reserve_students.pop(0)  # Берем первого студента из резерва
                        free_students.append(student_from_reserve)  # Добавляем студента во свободные списки
        else:
            # Если студент сделал все предложения, он становится несопоставленным
            free_students.pop(0)

    # Список студентов с пустыми предпочтениями
    students_with_no_preferences = [s for s in range(num_students) if len(preferences_s[s]) == 0]
    # Основной цикл распределения завершен

    # Теперь распределяем студентов с пустыми предпочтениями случайным образом
    for student in students_with_no_preferences:
        available_profiles = [prof for prof in range(num_profiles) if len(profiles_matches[prof]) < quotas[prof]]
        if available_profiles:  # Если есть доступные профили
            prof = random.choice(available_profiles)  # Случайный профиль
            profiles_matches[prof].append(student)  # Записываем студента на профиль
            student_matches[student] = prof  # Обновляем сопоставление студента

    # Список несопоставленных студентов
    unmatched_students = [s for s in range(num_students) if student_matches[s] is None]

    return profiles_matches, student_matches, unmatched_students

In [4]:
profiles_matches, student_matches, unmatched_students = gale_shapley(
    num_students, num_profiles, quotas, preferences_s, preferences_p, mandatory_profiles, reserve_students
)

for prof, matched_students in profiles_matches.items():
    print(f"Профиль {prof}: Студенты - {matched_students}")
    print(f"Всего на профиле {prof} {len(matched_students)} студентов")

print("Незачисленные студенты:")
print(unmatched_students)

Профиль 0: Студенты - [382, 249, 177, 98, 330, 360, 0, 62, 446, 107, 373, 339, 334, 140, 290, 396, 348, 207, 103, 355, 51, 436, 101, 128, 376, 381, 286, 71, 352, 302, 192, 18, 256, 358, 215, 125, 412, 109, 134, 248, 118, 86, 392, 19, 94, 203, 398, 218, 72, 1, 96, 323]
Всего на профиле 0 52 студентов
Профиль 1: Студенты - [164, 394, 368, 442, 130, 46, 153, 310, 433, 425, 83, 54, 42, 43, 258, 65, 50, 434, 359, 144, 106, 22, 26, 391, 415, 73, 44, 414, 282, 338, 127, 265, 315, 246, 400, 335, 154, 89, 9, 181, 52, 432, 387, 206, 195, 233, 30, 447, 318, 39, 15, 284, 61, 145, 189, 361, 366]
Всего на профиле 1 57 студентов
Профиль 2: Студенты - [221, 76, 291, 110, 35, 301, 262, 175, 82, 45, 408, 59, 147, 168, 395, 21, 67, 17, 378, 57, 407, 257, 113, 25, 201, 255, 245, 224, 317, 309, 329, 129, 371, 243, 423, 24, 13, 430, 81, 241, 174, 242, 272, 420, 204, 226, 269, 143, 362, 214, 108, 180, 33, 68, 172, 236, 340, 404, 448]
Всего на профиле 2 59 студентов
Профиль 3: Студенты - [7, 235, 419, 289, 18